In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imp
import core
import utils
%matplotlib inline
wd = 'E:/new_data/kaggle/planet/'
train_set = pd.read_csv(wd + 'train_v2.csv')
train_set['tags'] = train_set['tags'].apply(lambda x: x.split(' '))
test_set = pd.read_csv(wd+'sample_submission_v2.csv')
train_tags = ['clear', 'partly_cloudy', 'haze', 'cloudy', 'primary', 'agriculture', 'road', 'water',
             'cultivation', 'habitation', 'bare_ground', 'selective_logging', 'artisinal_mine', 
              'blooming', 'slash_burn', 'conventional_mine', 'blow_down']
label_map = {l: i for i, l in enumerate(train_tags)}
inv_label_map = {i: l for l, i in label_map.items()}
file_all = train_set['image_name'].values
y_tr = utils.get_y(train_set['tags'].values, label_map)
test_file_all = test_set['image_name'].values

nnts = np.load('../preds/170620_01_pred_ts.npy')
nntr = np.load('../preds/170620_01_pred_tr.npy')

gbts = np.load('../preds/gbm_all_pred_ts_170624_01.npy')
gbtr = np.load('../preds/gbm_all_pred_tr_170624_01.npy')

wnnts = np.load('./pred/modelweightedNN1_date710_pred_test.npy')
wnntr = np.load('./pred/modelweightedNN1_date710_pred_train.npy')

nbtr = np.load("./pred/modelnn_boost_date716_pred_train.npy")
nbts = np.load("./pred/modelnn_boost_date716_pred_test.npy")

rnnts = np.load('./pred/modelresnet1_date715_pred_test.npy')
rnntr = np.load('./pred/modelresnet1_date715_pred_train.npy')

In [2]:
nntr.shape, nnts.shape, gbtr.shape, gbts.shape, wnntr.shape, wnnts.shape, nbtr.shape, nbts.shape, y_tr.shape

((40479, 17),
 (61191, 17),
 (40479, 17),
 (61191, 17),
 (40479, 17),
 (61191, 17),
 (40479, 17),
 (61191, 17),
 (40479, 17))

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import fbeta_score
from sklearn.model_selection import StratifiedKFold

y_ts = np.zeros((gbts.shape[0], 17))
p_tr = np.zeros((gbtr.shape[0], 17))

for i_c in range(17):
    skf = StratifiedKFold(n_splits=5)
    x_tr = np.c_[nntr[:, i_c], gbtr[:, i_c], wnntr[:, i_c], nbtr[:, i_c], rnntr[:, i_c]]
    x_ts = np.c_[nnts[:, i_c], gbts[:, i_c], wnnts[:, i_c], nbts[:, i_c], rnnts[:, i_c]]
    k_now = 0
    for i_tr, i_vl in skf.split(x_tr, y_tr[:, i_c]):
        lr = LogisticRegression(class_weight='balanced')
        lr.fit(x_tr[i_tr, :], y_tr[i_tr, i_c])
        p_tr[i_vl, i_c] = lr.predict_proba(x_tr[i_vl, :])[:,1]

    lr = LogisticRegression(class_weight='balanced')
    lr.fit(x_tr, y_tr[:, i_c])
    y_ts[:, i_c] = lr.predict_proba(x_ts)[:, 1]

In [8]:
th1 = utils.f2_opti_score(y_tr, p_tr, thresholds = np.arange(0, 1, 0.01))
th2 = utils.f2_opti_score(y_tr, p_tr, thresholds = np.arange(1, 0, -0.01))
th = (th1 + th2) / 2.0
utils.f2_score(y_tr, p_tr, th, num_classes=17)

0.92184654319648474

In [5]:
imp.reload(utils)
submit_df = utils.to_submit(y_ts, th, test_set, inv_label_map)
submit_df.to_csv('./submit/4model_stacking_date716_no2.csv', index=False)

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

y_ts = np.zeros((gbts.shape[0], 17))
p_tr = np.zeros((gbtr.shape[0], 17))

for i_c in range(17):
    x_tr = np.c_[nntr[:, i_c], gbtr[:, i_c], wnntr[:, i_c], nbtr[:, i_c], rnntr[:, i_c]]
    x_ts = np.c_[nnts[:, i_c], gbts[:, i_c], wnnts[:, i_c], nbts[:, i_c], rnnts[:, i_c]]
    parameters = {'n_neighbors': range(3, 61, 3)}
    neigh = KNeighborsClassifier()
    clf = GridSearchCV(neigh, parameters, n_jobs=4)
    clf.fit(x_tr, y_tr[:, i_c])
    print('i: {}, Best score: {}, Best param {}.'.format(i_c, clf.best_score_, clf.best_params_['n_neighbors']))
    neigh = KNeighborsClassifier(n_neighbors=clf.best_params_['n_neighbors'])
    neigh.fit(x_tr, y_tr[:, i_c])
    y_ts[:, i_c] = neigh.predict_proba(x_ts)[:, 1]

Best score: 0.9565453692037846, Best param 60.
Best score: 0.9725042614689098, Best param 33.
Best score: 0.9649694903530226, Best param 54.
Best score: 0.9834482077126411, Best param 48.
Best score: 0.9704044072234986, Best param 45.
Best score: 0.9037031547222016, Best param 60.
Best score: 0.9284814348180538, Best param 57.
Best score: 0.9254181180365127, Best param 48.
Best score: 0.9208478470317942, Best param 57.
Best score: 0.9503693273055165, Best param 60.
Best score: 0.9798413992440524, Best param 36.
Best score: 0.9920946663702167, Best param 12.
Best score: 0.9965661207045629, Best param 12.
Best score: 0.9919217371970651, Best param 9.
Best score: 0.9948368289730477, Best param 6.
Best score: 0.9982707082684849, Best param 21.
Best score: 0.997653104078658, Best param 24.


In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

y_ts = np.zeros((gbts.shape[0], 17))
p_tr = np.zeros((gbtr.shape[0], 17))

n_neighbors = [60, 33, 54, 48,45,60,57,48,57,60,36,12,12,9,6,21,24]
for i_c in range(17):
    skf = StratifiedKFold(n_splits=5)
    x_tr = np.c_[nntr[:, i_c], gbtr[:, i_c], wnntr[:, i_c], nbtr[:, i_c], rnntr[:, i_c]]
    x_ts = np.c_[nnts[:, i_c], gbts[:, i_c], wnnts[:, i_c], nbts[:, i_c], rnnts[:, i_c]]
    k_now = 0
    for i_tr, i_vl in skf.split(x_tr, y_tr[:, i_c]):
        neigh = KNeighborsClassifier(n_neighbors=n_neighbors[i_c])
        neigh.fit(x_tr[i_tr, :], y_tr[i_tr, i_c])
        p_tr[i_vl, i_c] = neigh.predict_proba(x_tr[i_vl, :])[:,1]
        
    neigh = KNeighborsClassifier(n_neighbors=n_neighbors[i_c])
    neigh.fit(x_tr, y_tr[:, i_c])
    y_ts[:, i_c] = neigh.predict_proba(x_ts)[:, 1]

In [8]:
th1 = utils.f2_opti_score(y_tr, p_tr, thresholds = np.arange(0, 1, 0.01))
th2 = utils.f2_opti_score(y_tr, p_tr, thresholds = np.arange(1, 0, -0.01))
th = (th1 + th2) / 2.0
utils.f2_score(y_tr, p_tr, th, num_classes=17)

J:\Anaconda3\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


0.9244672299854505

In [13]:
submit_df = utils.to_submit(y_ts, th * 0.95, test_set, inv_label_map)
submit_df.to_csv('./submit/5model_stackingwithknn_date716_no4.csv', index=False)

In [12]:
th * 0.95

array([ 0.16625,  0.133  ,  0.14725,  0.09025,  0.1805 ,  0.16625,
        0.15675,  0.20425,  0.19475,  0.21375,  0.171  ,  0.1995 ,
        0.2755 ,  0.3705 ,  0.7125 ,  0.114  ,  0.2565 ])